# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [27]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [28]:
!cat intlpresentation.tsv

category	date	title	venue	location	talk_url	venue_url	description
Presentation	2021-07-11	The Edge Interdiction Clique Problem	EURO Athens	Greece		https://euro2021athens.com/	
Presentation	2020-10-20	Two Branch-and-cut Algorithms for the Directed Network Design Problem with Relays	15th INFORMS Telecommunications and Network Analytics Conference	Online		http://informs_tnac2020.zib.de	
Presentation	2019-10-20	A New Bilevel Approach For the k-vertex Cut Problem	INFORMS Annual Meeting	Seattle, Washington, US		http://meetings2.informs.org/wordpress/seattle2019/	
Presentation	2019-06-12	On Integer and Bilevel Formulations for the k-Vertex Cut Problem	International Network Optimization Conference	Avignon, France	slides/INOC2019_Ljubic.pdf	https://inoc2019.sciencesconf.org/	
Presentation	2019-06-02	Optimal vehicle routing with autonomous devices for last-mile delivery	VeRoLog - Workshop of the EURO Working Group on Vehicle Routing and Logistics optimization	Seville, Spain		https://verolog2019.

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [42]:
talks = pd.read_csv("intlpresentation.tsv", sep="\t", header=0)
talks

,category,date,title,venue,location,talk_url,venue_url,description
0,Presentation,2021-07-11,The Edge Interdiction Clique Problem,EURO Athens,Greece,NaN,https://euro2021athens.com/,NaN
1,Presentation,2020-10-20,Two Branch-and-cut Algorithms for the Directed...,15th INFORMS Telecommunications and Network An...,Online,NaN,http://informs_tnac2020.zib.de,NaN
2,Presentation,2019-10-20,A New Bilevel Approach For the k-vertex Cut Pr...,INFORMS Annual Meeting,"Seattle, Washington, US",NaN,http://meetings2.informs.org/wordpress/seattle...,NaN
3,Presentation,2019-06-12,On Integer and Bilevel Formulations for the k-...,International Network Optimization Conference,"Avignon, France",slides/INOC2019_Ljubic.pdf,https://inoc2019.sciencesconf.org/,NaN
4,Presentation,2019-06-02,Optimal vehicle routing with autonomous device...,VeRoLog - Workshop of the EURO Working Group o...,"Seville, Spain",NaN,https://verolog2019.sciencesconf.org/browse/sp...,NaN
5,Presentation,2018-11-02,Clique Interdiction Attacks in Networks,INFORMS Annual Meeting,"Phoenix, AZ, US",NaN,http://meetings2.informs.org/wordpress/phoenix...,NaN
6,Presentation,2018-07-09,Benders Decomposition for Covering Location Pr...,EURO 2018,"Valencia, Spain",NaN,http://euro2018valencia.com/,NaN
7,Presentation,2018-07-03,Decomposition Approaches to Covering Location ...,ISMP 2018,"Bordeaux, France",NaN,NaN,NaN
8,Presentation,2018-06-08,The Directed Network Design Problem with Relays,Odysseus - Seventh International Workshop on F...,"Cagliari, Italy",slides/Odysseus2018_Ljubic.pdf,http://convegni.unica.it/odysseus2018/,NaN
9,Presentation,2018-01-01,MIP Modeling of Incremental Connected Facility...,INFORMS Telecommunications Conference,"Hamburg, Germany",slides/informsTelecom2018_Ljubic.pdf,https://www.bwl.uni-hamburg.de/en/iwi/forschun...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [43]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [44]:
loc_dict = {}

for row, item in talks.iterrows():
    url_slug = '-'.join([i for i in item.title.strip().lower().split()])
    
    md_filename = str(item.date) + "-" + url_slug + ".md"
    html_filename = str(item.date) + "-" + url_slug 
    year = item.date[:4]
    html_filename = html_filename.replace(',','')
    md_filename = md_filename.replace(',','')
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.category)) > 3:
        md += 'category: "' + item.category + '"\n'
    else:
        md += 'category: "Talk"\n'
    
    md += 'type: "intlpresentation"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    print("permalink: /talks/" + html_filename + "\n")
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    if len(str(item.venue)) > 3:
        if len(str(item.venue_url)) > 3:
            md += "\nThis talk was part of [" + item.venue + "](" + item.venue_url + ")" 
        else:
            md += "\nThis talk was part of " + item.venue

        if len(str(item.talk_url)) > 3:
            if 'slides' in str(item.talk_url):
                md += " and more information can be found [here]({{site.url}}/docs/" + item.talk_url + ")\n" 
            else:
                md += " and more information can be found [here](" + item.talk_url + ")\n" 
        else:
            md += "."
    else:
        if len(str(item.talk_url)) > 3:
            md += "\nMore information can be found [here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

permalink: /talks/2021-07-11-the-edge-interdiction-clique-problem

permalink: /talks/2020-10-20-two-branch-and-cut-algorithms-for-the-directed-network-design-problem-with-relays

permalink: /talks/2019-10-20-a-new-bilevel-approach-for-the-k-vertex-cut-problem

permalink: /talks/2019-06-12-on-integer-and-bilevel-formulations-for-the-k-vertex-cut-problem

permalink: /talks/2019-06-02-optimal-vehicle-routing-with-autonomous-devices-for-last-mile-delivery

permalink: /talks/2018-11-02-clique-interdiction-attacks-in-networks

permalink: /talks/2018-07-09-benders-decomposition-for-covering-location-problems

permalink: /talks/2018-07-03-decomposition-approaches-to-covering-location-problems

permalink: /talks/2018-06-08-the-directed-network-design-problem-with-relays

permalink: /talks/2018-01-01-mip-modeling-of-incremental-connected-facility-location

permalink: /talks/2017-09-22-decomposition-methods-for-stochastic-steiner-trees

permalink: /talks/2017-09-07-outer-approximation-and-submodu

These files are in the talks directory, one directory below where we're working from.

In [15]:
!ls ../_talks

2006-07-31-optimization-in-telecommunication-networks:-multicommodity-flow,-facility-location-and-steiner-tree-problems.md
2007-03-19-connected-facility-location-in-the-design-of-telecommunication-networks.md
2010-06-27-two-stage-branch-&-cut-for-two-stage-stochastic-network-design-problems.md
2010-08-16-dissaggregated-flow-formulation-for-the-single-source-network-loading-problem.md
2010-08-16-opttelnets:-algorithmische-ansätze.md
2011-10-21-the-generalized-regenerator-location-problem.md
2012-01-27-solving-two-stage-stochastic-steiner-tree-problems-by-two-stage-branch-and-cut.md
2012-04-04-optimization-tools-for-last-mile-access-networks.md
2012-04-26-the-recoverable-robust-two-level-network-design-problem.md
2012-07-04-solving-2-stage-stochastic-network-design-problems-by-2-stage-branch-and-cut.md
2012-11-22-the-maximum-weight-connected-subgraph-problem.md
2013-08-29-the-maximum-weight-connected-subgraph-problem:-applications-in-bioinformatics.md
2014-05-13-the-recoverable-robust-fa

In [15]:
!cat ../_talks/2012-03-01-mixed-integer-bilevel-optimization.md

---
title: "Mixed Integer Bilevel Optimization"
collection: talks
type: "Talk"
permalink: /talks/2012-03-01-mixed-integer-bilevel-optimization
venue: "SIAM Conference on Optimization (OP21)"
---
This talk was part of [SIAM Conference on Optimization (OP21)](https://www.siam.org/conferences/cm/program/invited-presentations/op21-invited-presentations) and more information can be found [here](https://meetings.siam.org/sess/dsp_programsess.cfm?SESSIONCODE=72554)
